# Assignment 4

In [44]:
import ml_pipeline as pp
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

## Load data

In [28]:
filename = './data/projects_2012_2013.csv'
df = pp.load_csv(filename)

## Process data for clustering

### Missing values

In [29]:
pp.na_col(df)

['school_ncesid',
 'school_metro',
 'school_district',
 'primary_focus_subject',
 'primary_focus_area',
 'secondary_focus_subject',
 'secondary_focus_area',
 'resource_type',
 'grade_level',
 'students_reached']

In [30]:
cat_col = ['school_metro',
 'school_district',
 'primary_focus_subject',
 'primary_focus_area',
 'secondary_focus_subject',
 'secondary_focus_area',
 'resource_type',
 'grade_level']
for c in cat_col:
    df = pp.na_fill_col(df, c, pp.most_freq)

In [31]:
df = pp.na_fill_col(df, 'students_reached', np.nanmean)

### Categorical variables to binary

In [32]:
df = pp.feat_binary(df, ['school_metro','school_charter','primary_focus_subject','primary_focus_area', 'grade_level', 'resource_type'])

### Normalization

In [35]:
num_col = ['total_price_including_optional_support', 'students_reached']

In [36]:
pp.normalize_mult_minmax(df, num_col)

MinMaxScaler(copy=True, feature_range=(0, 1))

### Given a k, generate k clusters using KMeans

In [39]:
list(df.columns)

['projectid',
 'teacher_acctid',
 'schoolid',
 'school_ncesid',
 'school_latitude',
 'school_longitude',
 'school_city',
 'school_state',
 'school_district',
 'school_county',
 'school_magnet',
 'teacher_prefix',
 'secondary_focus_subject',
 'secondary_focus_area',
 'poverty_level',
 'total_price_including_optional_support',
 'students_reached',
 'eligible_double_your_impact_match',
 'date_posted',
 'datefullyfunded',
 'school_metro_rural',
 'school_metro_suburban',
 'school_metro_urban',
 'school_charter_f',
 'school_charter_t',
 'primary_focus_subject_Applied Sciences',
 'primary_focus_subject_Character Education',
 'primary_focus_subject_Civics & Government',
 'primary_focus_subject_College & Career Prep',
 'primary_focus_subject_Community Service',
 'primary_focus_subject_ESL',
 'primary_focus_subject_Early Development',
 'primary_focus_subject_Economics',
 'primary_focus_subject_Environmental Science',
 'primary_focus_subject_Extracurricular',
 'primary_focus_subject_Foreign Langu

In [40]:
features = ['school_metro_rural',
 'school_metro_suburban',
 'school_metro_urban',
 'school_charter_f',
 'school_charter_t',
 'primary_focus_subject_Applied Sciences',
 'primary_focus_subject_Character Education',
 'primary_focus_subject_Civics & Government',
 'primary_focus_subject_College & Career Prep',
 'primary_focus_subject_Community Service',
 'primary_focus_subject_ESL',
 'primary_focus_subject_Early Development',
 'primary_focus_subject_Economics',
 'primary_focus_subject_Environmental Science',
 'primary_focus_subject_Extracurricular',
 'primary_focus_subject_Foreign Languages',
 'primary_focus_subject_Gym & Fitness',
 'primary_focus_subject_Health & Life Science',
 'primary_focus_subject_Health & Wellness',
 'primary_focus_subject_History & Geography',
 'primary_focus_subject_Literacy',
 'primary_focus_subject_Literature & Writing',
 'primary_focus_subject_Mathematics',
 'primary_focus_subject_Music',
 'primary_focus_subject_Nutrition',
 'primary_focus_subject_Other',
 'primary_focus_subject_Parent Involvement',
 'primary_focus_subject_Performing Arts',
 'primary_focus_subject_Social Sciences',
 'primary_focus_subject_Special Needs',
 'primary_focus_subject_Sports',
 'primary_focus_subject_Visual Arts',
 'primary_focus_area_Applied Learning',
 'primary_focus_area_Health & Sports',
 'primary_focus_area_History & Civics',
 'primary_focus_area_Literacy & Language',
 'primary_focus_area_Math & Science',
 'primary_focus_area_Music & The Arts',
 'primary_focus_area_Special Needs',
 'grade_level_Grades 3-5',
 'grade_level_Grades 6-8',
 'grade_level_Grades 9-12',
 'grade_level_Grades PreK-2',
 'resource_type_Books',
 'resource_type_Other',
 'resource_type_Supplies',
 'resource_type_Technology',
 'resource_type_Trips',
 'resource_type_Visitors',
    'total_price_including_optional_support', 'students_reached']

In [46]:
X = df[features]
for n_clusters in [2,3,4]:
    kmeans = KMeans(n_clusters=n_clusters).fit(X)
    X['pred_label'] = pd.Series(kmeans.labels_)

    groups = X.groupby('pred_label')

    print(groups)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [48]:
kmeans = KMeans(n_clusters=2).fit(X)
X['pred_label'] = pd.Series(kmeans.labels_)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
one = X[X['pred_label'] == 1]
zero = X[X['pred_label'] == 0]

In [72]:
res =pd.DataFrame(one.sum(axis=0)/one.shape[0])
res.columns = ['percent']
res.sort_values('percent', ascending = False)

,percent
pred_label,1.000000
school_charter_f,0.908649
school_metro_urban,0.614294
primary_focus_area_Math & Science,0.460078
resource_type_Supplies,0.420171
resource_type_Technology,0.374561
grade_level_Grades 3-5,0.312596
grade_level_Grades PreK-2,0.308838
school_metro_suburban,0.266507
primary_focus_subject_Mathematics,0.244652


In [74]:
res2 =pd.DataFrame(zero.sum(axis=0)/zero.shape[0])
res2.columns = ['percent']
res2.sort_values('percent', ascending=False)

,percent
primary_focus_area_Literacy & Language,1.000000
school_charter_f,0.904737
primary_focus_subject_Literacy,0.667481
school_metro_urban,0.627118
grade_level_Grades PreK-2,0.471783
resource_type_Technology,0.358363
resource_type_Books,0.342971
grade_level_Grades 3-5,0.315717
primary_focus_subject_Literature & Writing,0.287893
school_metro_suburban,0.262530
